In [73]:
# !pip -qq install torchtext==0.3.1
# !git clone https://github.com/MiuLab/SlotGated-SLU.git
# !wget -qq https://raw.githubusercontent.com/yandexdataschool/nlp_course/master/week08_multitask/conlleval.py

In [3]:
import numpy as np

import torch, random
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

if torch.cuda.is_available():
    from torch.cuda import FloatTensor, LongTensor
    DEVICE = torch.device('cuda:0')
else:
    from torch import FloatTensor, LongTensor
    DEVICE = torch.device('cpu')

SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

DEVICE

device(type='cuda', index=0)

In [4]:
import os 

def read_dataset(path):
    with open(os.path.join(path, 'seq.in')) as f_words, \
            open(os.path.join(path, 'seq.out')) as f_tags, \
            open(os.path.join(path, 'label')) as f_intents:
        
        return [
            (words.strip().split(), tags.strip().split(), intent.strip()) 
            for words, tags, intent in zip(f_words, f_tags, f_intents)
        ]
train_data = read_dataset('SlotGated-SLU/data/atis/train/')
val_data = read_dataset('SlotGated-SLU/data/atis/valid/')
test_data = read_dataset('SlotGated-SLU/data/atis/test/')

intent_to_example = {example[2]: example for example in train_data}
# for example in intent_to_example.values():
#     print('Intent:\t', example[2])
#     print('Text:\t', '\t'.join(example[0]))
#     print('Tags:\t', '\t'.join(example[1]))
#     print()
    
from torchtext.data import Field, LabelField, Example, Dataset, BucketIterator

tokens_field = Field()
tags_field = Field(unk_token=None)
intent_field = LabelField()

fields = [('tokens', tokens_field), ('tags', tags_field), ('intent', intent_field)]

train_dataset = Dataset([Example.fromlist(example, fields) for example in train_data], fields)
val_dataset = Dataset([Example.fromlist(example, fields) for example in val_data], fields)
test_dataset = Dataset([Example.fromlist(example, fields) for example in test_data], fields)

tokens_field.build_vocab(train_dataset)
tags_field.build_vocab(train_dataset)
intent_field.build_vocab(train_dataset)

print('Vocab size =', len(tokens_field.vocab))
print('Tags count =', len(tags_field.vocab))
print('Intents count =', len(intent_field.vocab))

train_iter, val_iter, test_iter = BucketIterator.splits(
    datasets=(train_dataset, val_dataset, test_dataset), batch_sizes=(32, 128, 128), 
    shuffle=True, device=DEVICE, sort=False
)

Vocab size = 869
Tags count = 121
Intents count = 21


In [5]:
class SharedModel(nn.Module):
    def __init__(self, vocab_size, intents_count, tags_count, emb_dim=64, lstm_hidden_dim=128, num_layers=1,  dropout_p=0.25, pad_idx = 0):
        super().__init__()

        self.embeddings_layer = nn.Embedding(vocab_size, emb_dim, padding_idx = pad_idx)
        self.dropout = nn.Dropout(dropout_p)
        self._intents_lstm_layer = nn.LSTM(input_size=emb_dim, hidden_size=lstm_hidden_dim, bidirectional=True, num_layers=num_layers, batch_first=True, dropout = dropout_p if num_layers > 1 else 0)
        self._tags_lstm_layer = nn.LSTM(input_size=emb_dim, hidden_size=lstm_hidden_dim, bidirectional=True, num_layers=num_layers, batch_first=True, dropout = dropout_p if num_layers > 1 else 0)
        self._intents_out_layer = nn.Linear(lstm_hidden_dim * 2, intents_count)
        self._tags_out_layer = nn.Linear(lstm_hidden_dim * 2, tags_count)       

    def forward(self, inputs):
        projections = self.dropout(self.embeddings_layer(inputs))
        output_lstm_intents, (final_hidden_state_intents, _) = self._intents_lstm_layer(projections)
        output_lstm_tags, (final_hidden_state_tags, _) = self._tags_lstm_layer(projections)
        
        hidden = self.dropout(torch.cat([final_hidden_state_intents[0], final_hidden_state_intents[1]], dim=1))
        output_intents = self._intents_out_layer(hidden)
        
        output_tags = self.dropout(output_lstm_tags)
        output_tags = self._tags_out_layer(output_tags)
        return (output_tags, output_intents)



In [6]:
class ModelTrainer_Trigger():
    def __init__(self, model, criterion, criterion_ignore_pad, optimizer_tags, optimizer_intent, pad_idx):
        self.model = model
        self.criterion = criterion
        self.criterion_ignore_pad = criterion_ignore_pad
        self.optimizer_tags = optimizer_tags
        self.optimizer_intent = optimizer_intent
        self.pad_idx = pad_idx
        self.best_valid_loss = float('inf')
        
    def on_epoch_begin(self, is_train, name, batches_count):
        """
        Initializes metrics
        """
        self.epoch_loss = 0
        self.correct_count, self.total_count = 0, 0
        self.is_train = is_train
        self.name = name
        self.batches_count = batches_count
        
        self.model.train(is_train)
        
    def on_epoch_end(self):
        """
        Outputs final metrics
        """
        valid_loss = self.epoch_loss / self.batches_count
        if not(self.is_train) and valid_loss < self.best_valid_loss:
            self.best_valid_loss = valid_loss
            torch.save(self.model, 'AsynMultiTaskModel.pt')
        return '{:>5s} Loss = {:.5f}, Accuracy = {:.2%}'.format(
            self.name, self.epoch_loss / self.batches_count, self.correct_count / self.total_count
        )
        
    def on_batch(self, batch):
        """
        Performs forward and (if is_train) backward pass with optimization, updates metrics
        """
        
        logits = self.model(batch.tokens.transpose(0, 1))
        
        logits_intents = logits[1]
        loss = self.criterion(logits_intents, batch.intent)
        
        if self.is_train:
            loss.backward(retain_graph = True)
            self.optimizer_intent.step()
            self.optimizer_intent.zero_grad()
        self.epoch_loss += loss.item()
        
        predicted_intent = logits_intents.argmax(dim=1)
        self.total_count += predicted_intent.size(0)
        self.correct_count += torch.sum(predicted_intent == batch.intent).item()
        

        logits_tags = logits[0].transpose(0, 1)  
        logits_tags = logits_tags.reshape((-1,logits_tags.shape[2]))
        tags = batch.tags.reshape((-1))
        loss = self.criterion_ignore_pad(logits_tags, tags)
        
        if self.is_train:
            loss.backward(retain_graph = True)
            self.optimizer_tags.step()
            self.optimizer_tags.zero_grad()
        self.epoch_loss += loss.item()


      
        max_preds = logits_tags.argmax(dim = 1) 
        non_pad_elements = (tags != self.pad_idx).nonzero()
        correct = torch.sum(max_preds[non_pad_elements] == tags[non_pad_elements]).item()
        self.correct_count += correct
        self.total_count += tags[non_pad_elements].shape[0]



        
import math
from tqdm import tqdm
tqdm.get_lock().locks = []

def do_epoch(trainer, data_iter, is_train, name=None):
    trainer.on_epoch_begin(is_train, name, batches_count=len(data_iter))
    
    with torch.autograd.set_grad_enabled(is_train):
        with tqdm(total=trainer.batches_count) as progress_bar:
            for i, batch in enumerate(data_iter):
                batch_progress = trainer.on_batch(batch)

                progress_bar.update()
                progress_bar.set_description(batch_progress)
                
            epoch_progress = trainer.on_epoch_end()
            progress_bar.set_description(epoch_progress)
            progress_bar.refresh()

            
def fit(trainer, train_iter, epochs_count=1, val_iter=None):
    best_val_loss = None
    for epoch in range(epochs_count):
        name_prefix = '[{} / {}] '.format(epoch + 1, epochs_count)
        do_epoch(trainer, train_iter, is_train=True, name=name_prefix + 'Train:')
        
        if not val_iter is None:
            do_epoch(trainer, val_iter, is_train=False, name=name_prefix + '  Val:')

In [7]:
tokens_pad_idx = tokens_field.vocab.stoi[tokens_field.pad_token]
tags_pad_idx = tags_field.vocab.stoi[tags_field.pad_token]

model = SharedModel(vocab_size=len(tokens_field.vocab), intents_count=len(intent_field.vocab), tags_count=len(tags_field.vocab), pad_idx = tokens_pad_idx).to(DEVICE)

tags_parameters = [param for name, param in model.named_parameters() if not name.startswith('_intent')]
intent_parameters = [param for name, param in model.named_parameters() if not name.startswith('_tags')]
# print([name for name, param in model.named_parameters()])
# print(len([name for name, param in model.named_parameters()]))
# print(len(tags_parameters))
# print(len(intent_parameters))

criterion_ignore_pad = nn.CrossEntropyLoss(ignore_index=tags_pad_idx).to(DEVICE)
criterion = nn.CrossEntropyLoss().to(DEVICE)
optimizer_tags = optim.Adam(tags_parameters)
optimizer_intent = optim.Adam(intent_parameters)

trainer = ModelTrainer_Trigger(model, criterion, criterion_ignore_pad, optimizer_tags, optimizer_intent, tags_pad_idx)

fit(trainer, train_iter, epochs_count=30, val_iter=val_iter)


[1 / 30] Train: Loss = 2.66810, Accuracy = 70.12%: 100%|██████████| 140/140 [00:02<00:00, 69.65it/s]
[1 / 30]   Val: Loss = 1.44941, Accuracy = 83.52%: 100%|██████████| 4/4 [00:00<00:00, 61.54it/s]
[2 / 30] Train: Loss = 0.99398, Accuracy = 88.34%: 100%|██████████| 140/140 [00:01<00:00, 77.65it/s]
[2 / 30]   Val: Loss = 0.82354, Accuracy = 91.05%: 100%|██████████| 4/4 [00:00<00:00, 75.47it/s]
[3 / 30] Train: Loss = 0.62603, Accuracy = 92.79%: 100%|██████████| 140/140 [00:02<00:00, 65.03it/s]
[3 / 30]   Val: Loss = 0.54641, Accuracy = 93.97%: 100%|██████████| 4/4 [00:00<00:00, 37.74it/s]
[4 / 30] Train: Loss = 0.45622, Accuracy = 94.72%: 100%|██████████| 140/140 [00:02<00:00, 50.16it/s]
[4 / 30]   Val: Loss = 0.45780, Accuracy = 95.23%: 100%|██████████| 4/4 [00:00<00:00, 36.04it/s]
[5 / 30] Train: Loss = 0.33751, Accuracy = 96.07%: 100%|██████████| 140/140 [00:02<00:00, 50.78it/s]
[5 / 30]   Val: Loss = 0.35902, Accuracy = 96.15%: 100%|██████████| 4/4 [00:00<00:00, 40.40it/s]
[6 / 30] T

In [8]:
model = torch.load('AsynMultiTaskModel.pt')

do_epoch(trainer, test_iter, is_train=False, name='Test:')

Test: Loss = 0.36012, Accuracy = 97.69%: 100%|██████████| 7/7 [00:00<00:00, 45.46it/s]


In [12]:
from conlleval import evaluate

def eval_tagger(model, test_iter):
    pad_idx = tags_field.vocab.stoi[tags_field.pad_token]
    true_seqs, pred_seqs = [], []
    correct_count, total_count = 0, 0

    model.eval()
    with torch.no_grad():
        for batch in test_iter:
            pred = model.forward(batch.tokens.transpose(0, 1))
            
            predicted_intent = pred[1].argmax(dim=1)
            total_count += predicted_intent.size(0)
            correct_count += torch.sum(predicted_intent == batch.intent).item()
            
            pred = pred[0].transpose(0, 1)
            pred = pred.reshape((-1,pred.shape[2]))
            true = batch.tags.reshape((-1))
            
            max_preds = pred.argmax(dim = 1)
            non_pad_elements = (true != pad_idx).nonzero()
            true = list(map(lambda y: tags_field.vocab.itos[y], true[non_pad_elements].reshape((-1))))
            pred = list(map(lambda y: tags_field.vocab.itos[y], max_preds[non_pad_elements].reshape((-1))))
            
            
            # print(true[200:220])
            # print(pred[200:220])
            # print()
        
            pred = ' '.join(pred)
            true = ' '.join(true)
        
  
            pred_seqs.append(pred)
            true_seqs.append(true)
            
            
    print('Precision = {:.2f}%, Recall = {:.2f}%, F1 = {:.2f}%'.format(*evaluate(true_seqs, pred_seqs, verbose=False)))
    print('Intent accuracy = {:.2%}%'.format(correct_count / total_count))

eval_tagger(model, test_iter)   


Precision = 94.83%, Recall = 93.95%, F1 = 94.39%
Intent accuracy = 96.53%%
